In [84]:
from __future__ import print_function
from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
import os

# data
data_root = os.path.expanduser("~") + '/data/CSE255/'

# l1-norm
import cvxopt as co
from l1 import l1

# natural language processing
import nltk
import nltk.data
import string
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stemmer = nltk.stem.porter.PorterStemmer()
punctuation = set(string.punctuation)

In [26]:
# load all_data
start_time = time.time()
all_data = pickle.load(open(data_root + "all_data.pickle", "rb"))
print(time.time() - start_time)

17.7351691723


In [95]:
# split training and valid set
# all
all_size = len(all_data)

# train
train_size = 900000
# train_size = all_size # uncomment this to produce test
train_data = all_data[:train_size]

# valid
valid_size = 100000
valid_data = all_data[all_size - valid_size:]

In [96]:
# remove the outlier
# for i in reversed(range(train_size)):
#     if train_data[i]['helpful']['outOf'] > 5000:
#         train_data.pop(i)

In [125]:
# get global average
train_helpfuls = np.array([d['helpful']['nHelpful'] for d in train_data])
train_outofs =  np.array([d['helpful']['outOf'] for d in train_data])
train_avg_ratio = np.sum(train_helpfuls) / np.sum(train_outofs.astype(float))
print('avg helpfulness ratio', train_avg_ratio)

# linear search best ratio
def linear_search_ratio(helpfuls, outofs, search_range=(0.3, 1.0, 0.001)):
    alphas = np.arange(*search_range)
    errors = [get_mae(helpfuls, outofs * alpha) for alpha in alphas]
    optimal_alpha = alphas[np.argmin(errors)]
    return optimal_alpha

# training set global
train_helpfuls = np.array([d['helpful']['nHelpful'] for d in train_data])
train_outofs =  np.array([d['helpful']['outOf'] for d in train_data])
train_avg_ratio = linear_search_ratio(train_helpfuls, train_outofs, search_range=(0.3, 1.0, 0.001))
print('optimal helpfulness ratio', train_avg_ratio)

# get average for a user
users_outof = dict()
users_helpful = dict()

for d in train_data:
    user_id = d['reviewerID']
    users_outof[user_id] = users_outof.get(user_id, 0.0) + float(d['helpful']['outOf'])
    users_helpful[user_id] = users_helpful.get(user_id, 0.0) + float(d['helpful']['nHelpful'])
    
users_ratio = dict()
for user_id in users_outof:
    if users_outof[user_id] != 0:
        users_ratio[user_id] = users_helpful[user_id] / users_outof[user_id]
    else:
        users_ratio[user_id] = train_avg_ratio
        
# get average for a item
items_outof = dict()
items_helpful = dict()

for d in train_data:
    item_id = d['itemID']
    items_outof[item_id] = items_outof.get(item_id, 0.0) + float(d['helpful']['outOf'])
    items_helpful[item_id] = items_helpful.get(item_id, 0.0) + float(d['helpful']['nHelpful'])

items_ratio = dict()
for item_id in items_outof:
    if items_outof[item_id] != 0:
        items_ratio[item_id] = items_helpful[item_id] / items_outof[item_id]
    else:
        items_ratio[item_id] = train_avg_ratio

avg helpfulness ratio 0.770464430813
optimal helpfulness ratio 0.857


In [126]:
train_data[0]

{'category': [['Books']],
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I572782694',
 'rating': 5.0,
 'reviewText': 'favorite of the series...May not have been as steamy as some of the others...but the characters, their depth, and believability were amazing.  wanted to curl up with Devlin and make it all better(wink wink). an amazing series...found Laura Kate when I stumbled onto Hearts in Darkness(one of my all time faves)...this series ranks up there with my Kresley Cole and Gena Showalter favorites.',
 'reviewTime': '05 3, 2014',
 'reviewerID': 'U243261361',
 'summary': 'Loved it',
 'unixReviewTime': 1399075200}

In [132]:
# feature engineering
with open('betas.pickle') as f:
    beta_us, beta_is = pickle.load(f)
    
with open('train_ratio_list.pickle') as f:
    train_ratio_list = pickle.load(f)
    
with open(os.path.join(data_root, 'num_unique_word.feature')) as f:
    num_unique_word_dict = pickle.load(f)

# get date time statistics
def get_y_m_d(d):
    unix_time = d['unixReviewTime']
    y, m, d = datetime.datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d').split('-')
    y = int(y)
    m = int(m)
    d = int(d)
    return(y, m, d)

def get_feature_time(d):
    y, m, d = get_y_m_d(d)
    y = min(y, 2014)
    y = max(y, 1996)
    # 1996 [1,0,..,0] 2014 [0,0,...,0]
    y_feature = [0] * (2014 - 1996 + 1)
    y_feature[y - 1996] = 1
    # jan [1,0,...,0] dec [0,0,...,0]
    m_feature = [0] * 12
    m_feature[m - 1] = 1
    # date1 [1,0,...,0] date31 [0,0,...,0]
    d_feature = [0] * 31
    d_feature[d - 1] = 1
    # concatenate
    feature = y_feature[:-1] + m_feature[:-1] + d_feature[:-1]
    return feature

def get_num_uique_word(d):
    wordCount = defaultdict(int)
    for w in d["reviewText"].split():
        w = "".join([c for c in w.lower() if not c in punctuation])
        w = stemmer.stem(w)
        wordCount[w] += 1
    return len(wordCount)

def get_feature(d):
    user_id = d['reviewerID']
    item_id = d['itemID']
    
    feature = [1.0]
    feature += [num_unique_word_dict[user_id][item_id]]
    feature += [users_ratio[user_id], items_ratio[item_id]]
    feature += [float(d['rating']), abs(float(d['rating']) - 0.770464430813)]
    feature += get_feature_style(d)
    
    return feature

In [133]:
# utility functions
def get_mae(helpfuls, helpfuls_predict):
    return np.sum(np.fabs(helpfuls_predict - helpfuls.astype(float))) / helpfuls.shape[0]

def get_valid_mae(valid_data, theta, train_avg_ratio, users_ratio, items_ratio):
    helpfuls = np.array([float(d['helpful']['nHelpful']) for d in valid_data])
    helpfuls_predict = np.array([predict_helpful(d, theta, train_avg_ratio, users_ratio, items_ratio) for d in valid_data])
    return get_mae(helpfuls, helpfuls_predict)

# get [feature, label] from single datum
def get_feature_and_ratio_label(d, users_ratio, items_ratio):
    # check valid
    outof = float(d['helpful']['outOf'])
    if outof == 0:
        raise('out of cannot be 0 for ratio')

    # get feature and ratio
    feature = get_feature(d)
    ratio_label = float(d['helpful']['nHelpful']) / float(d['helpful']['outOf'])
    return (feature, ratio_label)

# build [feature, label] list from entire dataset
def make_average_regression_dataset(train_data, users_ratio, items_ratio):
    features = []
    labels = []
    
    for d in train_data:
        if float(d['helpful']['outOf']) == 0:
            continue
        feature, label = get_feature_and_ratio_label(d, users_ratio, items_ratio)
        features.append(feature)
        labels.append(label)
    return (np.array(features), np.array(labels))

# the official prediction function
def predict_helpful(d, theta, train_avg_ratio, users_ratio, items_ratio):
    user_id = d['reviewerID']
    item_id = d['itemID']
    outof = float(d['helpful']['outOf'])
    
    if (user_id in users_ratio) and (item_id in items_ratio):
        ratio = np.dot(get_feature(d), theta)
    elif (user_id in users_ratio) and (item_id not in items_ratio):
        ratio = users_ratio[user_id]
    elif (user_id not in users_ratio) and (item_id in items_ratio):
        ratio = items_ratio[item_id]
    else:
        ratio = train_avg_ratio
    return ratio * outof

In [134]:
# build dataset
train_xs, train_ys = make_average_regression_dataset(train_data, users_ratio, items_ratio)

In [144]:
import copy
train_xs_backup = np.zeros_like(train_xs)
train_xs_backup[:] = train_xs

train_xs = train_xs_backup[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]]

# [num_chars, num_words, num_sentences, redability, punctuation_ratio, capital_ratio, avg_word_len, exclam_count, dotdotdot_count]
#  6          7          8              9           10                 11             12            13            14

In [135]:
# solve 2-norm minimize problem
theta, residuals, rank, s = np.linalg.lstsq(train_xs, train_ys)

In [137]:
# solve 1-norm minimize problem
P = co.matrix(train_xs)
q = co.matrix(train_ys.reshape((train_ys.shape[0], 1)))
u = l1(P,q)
theta = np.array(u).reshape((-1,))
print(theta)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.1407e+05  2.9669e+04  8e+04  1e-17  4e+02  1e+00


ValueError: Rank(A) < p or Rank([G; A]) < n

In [ ]:
# validation result
print(get_valid_mae(valid_data, theta, train_avg_ratio, users_ratio, items_ratio))

In [ ]:
# ############ produce test ############

# # load helpful_data.json
# test_data = pickle.load(open(data_root + "helpful_data.pickle", "rb"))

# # on test set
# test_helpfuls_predict = [predict_helpful(d, theta, train_avg_ratio, users_ratio, items_ratio) for d in test_data]

# # load 'pairs_Helpful.txt'
# # get header_str and user_item_outofs
# with open('pairs_Helpful.txt') as f:
#     # read and strip lines
#     lines = [l.strip() for l in f.readlines()]
#     # stirip out the headers
#     header_str = lines.pop(0)
#     # get a list of user_item_ids
#     user_item_outofs = [l.split('-') for l in lines]
#     user_item_outofs = [[d[0], d[1], float(d[2])] for d in user_item_outofs]
    
# # make sure `data.json` and `pairs_Helpful.txt` the same order
# for (user_id, item_id, outof), d in zip(user_item_outofs, test_data):
#     assert d['reviewerID'] == user_id
#     assert d['itemID'] == item_id
#     assert d['helpful']['outOf'] == outof
    
# # write to output file
# f = open('predictions_Helpful.txt', 'w')
# print(header_str, file=f)
# for (user_id, item_id, outof), test_helpful_predict in zip(user_item_outofs, test_helpfuls_predict):
#     print('%s-%s-%s,%s' % (user_id, item_id, int(outof), test_helpful_predict), file=f)
# f.close()